In [ ]:
data = list(range(4))
def display():
        print("Data:", data)
        if __name__ == "__main__":
            display()
            print("This file was executed from the command line or an interpreter.")
        else:
            print("This file was imported.")
            
            

In [ ]:
def timeiter_prodshock():    
    numpts=30
    numapts=10
    mingrid=0.01
    maxgrid=10
    kgrid=np.linspace(mingrid,maxgrid,numpts)
    agrid=np.linspace(-1,1,numapts)
    diff=1
    ixiter=0
    V=np.empty([len(kgrid),len(kgrid)])
    pg=np.linspace(mingrid,maxgrid,numpts)
    policygrid,jk  = np.meshgrid(pg, agrid)
    policygrid=np.transpose(policygrid)
    policyguess1= policygrid
    policyguess2= policygrid
    Kprime=np.empty((len(kgrid),len(sgrid),len(agrid)))
    k1guess=mingrid*np.ones((len(kgrid),len(sgrid),len(agrid)))
    err=np.zeros((len(kgrid),len(sgrid),len(agrid)))
    for ixiter  in range(max_iterations): 
        EV = np.zeros(len(kgrid))
        for ii, K in enumerate(kgrid): 
            #K=kgrid[ki]
            for jj, S in enumerate(sgrid): 
                #S=sgrid[sj]
                for aa, A1 in enumerate(agrid):                
                    def eulereq(K1):
                        f, fder=fk(K,S,A1)
                        f1k1,f1der=fk(K1,1,A1)
                        prod = lambda eps: np.exp(0.9*A1+eps)
                        EZ, junk=scipy.integrate.quad(prod, -2, 2)                   
                        f1k1=EZ*K**0.3+0.5*K1
                        f1der=EZ+0.5*K1
                        f2k1, f2der=fk(K1,2,A1)
                        c=f-K1
                        cprime1=f1k1-np.interp(K1, policygrid[:,aa], policyguess1[:,aa])
                        cprime2=f2k1-np.interp(K1, policygrid[:,aa], policyguess2[:,aa])
                        ee=1/c-beta*pi*(f1der*(1/cprime1)+f2der*(1/cprime2)) 
                        ee[c<=0]=500
                        ee[cprime1<=0]=500
                        ee[cprime2<=0]=500
                        return ee
                    root=scipy.optimize.root(eulereq, k1guess[ii,jj,aa])
                    Kprime[ii,jj,aa]=root.x
                    err[ii,jj,aa]=eulereq(root.x)
#                print(err[ii,jj])
#                 Vvalue=objectivefun(EV1,K,K1,S)
#                 optval=np.amax(Vvalue)
#                 policy[ii,jj]=kgrid[np.argmax(Vvalue)]
#                 V[ii,jj]=optval              
#            EV[ii]=pi* V[ii,0]+pi* V[ii,1]
        diff=np.amax(abs(err))
    #    print(err)
        policyguess1=Kprime[:,0,:]
        policyguess2=Kprime[:,1,:]
        k1guess[ii,jj]=Kprime[ii,jj,aa]
        if diff < tolForFixedPoint:
            break
    print(ixiter)
    plt.plot(kgrid, policyguess1[:,0])
    plt.plot(kgrid, policyguess2[:,0])
    return policyguess1, policyguess2
